In [1]:
#imports
import numpy as np
import astropy 
from astropy.io import ascii
from astropy.table import Table, Column, join, vstack
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
%matplotlib inline

In [2]:
   #0) Check for good magnitudes
def magCheck():
    data[w1sig].fill_value = -999.0
    data[w2sig].fill_value = -999.0
    
    indices1 = (data[w1chi]<(data[w1sn]-3)/7.0).nonzero()
    indices2 = (data[w1sig][indices1] != -999.0).nonzero()
    goodMags = (data[w2sig][indices2] != -999.0).nonzero()
    
    return goodMags

#1) test for Star Forming Galaxies
def testSFG(indexArray):
    SFGs = []
    notSFGs = []
    for i in indexArray: 
        if (data[w2][i]-data[w3][i]> 2.3 
        and data[w1][i]-data[w2][i] < 1.0 
        and data[w1][i]> 13.0 
        and data[w1][i]-data[w2][i] < 0.46*(data[w2][i]-data[w3][i])-0.78):
            SFGs.append(i)
        else:
            notSFGs.append(i)
    return SFGs, notSFGs

#2) test for Active Galatic Nuclei
def testAGN(indexArray):
    AGNs = []
    notAGNs = []
    for i in indexArray:
        if(data[w1][i]>1.8*(data[w1][i]-data[w3][i])+4.1 
        and data[w1][i] >13.0 
        or data[w1][i]>data[w1][i]-data[w3][i]+11.0):
            AGNs.append(i)
        else:
            notAGNs.append(i)
        
    return AGNs, notAGNs

#3) test for Wise Class 1 YSOs
def testWISEClass1(indexArray):
    WISEClass1 = []
    notWISEClass1 = []
    for i in indexArray:
        if (data[w2][i]-data[w3][i] > 2.0 
        and (data[w1][i] - data[w2][i]) > -0.42*(data[w2][i] - data[w3][i])+ 2.2
        and (data[w1][i] - data[w2][i]) > 0.46*(data[w2][i] - data[w3][i])-0.9
        and data[w2][i] - data[w3][i] < 4.5):
            WISEClass1.append(i)
        else:
            notWISEClass1.append(i)
    return WISEClass1, notWISEClass1


#4) test for Wise Class 2 YSOs
def testWISEClass2(indexArray):
    WISEClass2 = []
    notWISEClass2 = []
    for i in indexArray:
        if (data[w1][i] - data[w2][i] > 0.25
        and data[w1][i] - data[w2][i] < 0.9*(data[w2][i] - data[w3][i]) - 0.25
        and data[w1][i] - data[w2][i] > -1.5*(data[w2][i] - data[w3][i]) + 2.1
        and data[w1][i] - data[w2][i] > 0.46*(data[w2][i] - data[w3][i]) - 0.9
        and data[w2][i] - data[w3][i] < 4.5):
            WISEClass2.append(i)
        else:
            notWISEClass2.append(i)
    return WISEClass2, notWISEClass2

#5) test for 2MASS YSO
def test2MASS(indexArray):
    YSO2MASS = []
    notYSO2MASS = []
    for i in indexArray:
        if ( not(data[w3sn][i] >=5.0 and 
            ((data[w3chi][i]> 0.45 and data[w3chi][i] < 1.15) 
            or data[w3chi][i]<(data[w3sn][i]-8)/8.0)) #fail w3 mag test
        and data[h][i] - data[k][i] > 0
        and data[h][i] - data[k][i] > -1.76*(data[w1][i] - data[w2][i])+0.9
        and data[h][i] - data[k][i] < (0.55/0.16)*(data[w1][i] - data[w2][i])-0.85
        and data[w1][i] <= 13.0):
            YSO2MASS.append(i)
        else:
            notYSO2MASS.append(i)     
    return YSO2MASS, notYSO2MASS

#6a) test for class 1's that were initially classified as class 2's
def test2MASSClass(indexArray):
    class1 = []
    class2 = []
    for i in indexArray:
        if (data[h][i] - data[k][i] > -1.76*(data[w1][i] - data[w2][i])+2.55):
            class1.append(i)
        else:
            class2.append(i)
    return class1, class2

#6b) test for Transition Disks
def testTransDisk(indexArray):
    transDisk = []
    noTransDisk = []
    data[w4sig].fill_value = -999.0
    for i in indexArray:
        if(data[w3][i] - data[w4][i] > 1.5
        and (0.15 < data[w1][i] - data[w2][i] and data[w1][i] - data[w2][i] < 0.8)
        and data[w1][i] - data[w2][i] > 0.46*(data[w2][i] - data[w3][i])-0.9
        and data[w1][i] <=13.0
        and data.filled()[w4sig][i] != -999.0 
        and data[w4chi][i]<(2*data[w4sn][i]-20)/10.0 #satisfy w4 test
        and (data[w3sn][i]>=5.0
        and((data[w3chi][i]> 0.45 and data[w3chi][i] < 1.15) 
        or data[w3chi][i]<(data[w3sn][i]-8)/8.0))
          ):  #satisfy w3 test
            transDisk.append(i)
        else:
            noTransDisk.append(i)
    return transDisk, noTransDisk

#6c) test for w4 class1 from AGNs
def testw4Class1(indexArray):
    class1 = []
    notClass1 = []
    data[w4sig].fill_value = -999.0
    for i in indexArray:
        if(data[w4][i]<5.0
        and (4.5 < data[w2][i] - data[w4][i] and data[w2][i] - data[w4][i] < 8.0)
        and data[w1][i] - data[w2][i] > 1.0
        and data[w3][i] - data[w4][i] > 2.0
        and data[w4chi][i]<(2*data[w4sn][i]-20)/10.0
        and data.filled()[w4sig][i] != -999.0
          ): #satisfy w4 mag test
            class1.append(i)
        else:
            notClass1.append(i)
    return class1, notClass1


#7) test for AGBs from all YSOs previously classified
def testAGB(indexArray):
    AGBs = []
    notAGBs = []
    data[w4sig].fill_value = -999.0
    for i in indexArray:
        if(data[w1][i] < (-10.0/3.0)*(data[w1][i] - data[w2][i])+9.0 
        or data[w1][i] < (6.0/7.0)*(data[w1][i] - data[w2][i])+5.5):
            AGBs.append(i)
        elif (data[w3][i] - data[w4][i] < 1.9 
        and data[w1][i] - data[w2][i] > data[w3][i] - data[w4][i] - 0.45 #flipped inequality?
        and data.filled()[w4sig][i] != -999.0 
        and data[w4chi][i]<(2*data[w4sn][i]-20)/10.0 #satisfy w4 test
        and (data[w3sn][i]>=5.0
        and((data[w3chi][i]> 0.45 and data[w3chi][i] < 1.15) 
        or data[w3chi][i]<(data[w3sn][i]-8)/8.0))
             ):  #satisfy w3 test
            AGBs.append(i)
        else:
            notAGBs.append(i)
    return AGBs, notAGBs

#---------------------------------------------------------------
start = 35
end = 37
while (start <= end):
    filename = '../../WISE/reducedWISETaurus/part'+str(start)
#read in the file
    data = ascii.read(filename)

#set names for magnitudes
    Esplin2MASS = False
    if Esplin2MASS:
        w1= 'w1mpro'
        w2= 'w2mpro'   
        w3= 'w3mpro' 
        w4='w4mpro' 

        w1chi='w1rchi2'  
        w2chi= 'w2rchi2'
        w3chi='w3rchi2'  
        w4chi=  'w4rchi2' 

        w1sn= 'w1snr'  
        w2sn='w2snr'
        w3sn='w3snr' 
        w4sn= 'w4snr'  

        w1sig='w1sigmpro' 
        w2sig='w2sigmpro'
        w3sig='3sigmpro'
        w4sig='w4sigmpro'

        h='h_m_2mass'
        k='k_m_2mass'
        j='j_m_2mass'

    else:
        w1= 'col17'  
        w2= 'col21'  
        w3= 'col25'
        w4= 'col29' 

        w1chi= 'col20'
        w2chi= 'col24'  
        w3chi= 'col28'  
        w4chi= 'col32'  

        w1sn= 'col19'
        w2sn='col23'  
        w3sn='col27'  
        w4sn= 'col28'  

        w1sig='col18'  
        w2sig='col22' 
        w3sig='col26'  
        w4sig='col30'  

        h='col290'
        k='col292'
        j='col288' 
    
    
#Each of the descisions are implemented. As indicated by the Koenig decision tree, 
#lists returned from each function are then passed to the next function as necessary.
    goodMags = magCheck()                                     #require good w1, w2
    SFGs, notSFGs = testSFG(goodMags[0])                      #check for SFGs
    AGNs, notAGNs = testAGN(notSFGs)                          #check for AGNs
    WISEClass1, notWISEClass1 = testWISEClass1(notAGNs)       #classify class 1
    WISEClass2, notWISEClass2 = testWISEClass2(notWISEClass1) #classify class 2
    YSO2MASS, notYSO2MASS = test2MASS(notWISEClass2)          #check for 2MASS YSOs
    Class12MASS, Class22MASS = test2MASSClass(YSO2MASS)       #define class of 2MASS YSOs
    TransDisk, noTransDisk = testTransDisk(notYSO2MASS)       #check for transition disks of sources that are not 2MASS YSOs
    w4Class1, notw4Class1 = testw4Class1(AGNs)                #check AGNs for w4 class 1

#check all classified YSOs for AGB characteristics
    WISEClass1AGB, WISEClass1YSO = testAGB(WISEClass1)
    WISEClass2AGB, WISEClass2YSO = testAGB(WISEClass2)
    Class12MASSAGB, Class12MASSYSO = testAGB(Class12MASS)
    Class22MASSAGB, Class22MASSYSO = testAGB(Class22MASS)
    TransDiskAGB, TransDiskYSO = testAGB(TransDisk)
    w4Class1AGB, w4Class1YSO = testAGB(w4Class1)

    allYSO = np.concatenate((WISEClass1YSO, WISEClass2YSO, Class12MASSYSO, Class22MASSYSO, TransDiskYSO, w4Class1YSO),axis=1)
    allAGB = np.concatenate((WISEClass1AGB, WISEClass2AGB, Class12MASSAGB, Class22MASSAGB, TransDiskAGB, w4Class1AGB),axis=1)

#create new columns for class and catalog
    colClass = Column(name = 'Class', dtype = 'a5', length = len(data))
    colGen = Column(name = 'GeneralSelection', dtype = 'a5', length = len(data))
    data.add_column(colClass)
    data.add_column(colGen)

#fill in new columns 
    for i in allYSO:
        data['GeneralSelection'][i] = 'K'
    
    for i in WISEClass1YSO:
        data['Class'][i] = 'I'
    
    for i in WISEClass2YSO:
        data['Class'][i] = 'II'

    for i in Class12MASSYSO:
        data['Class'][i] = 'I'
    
    for i in Class22MASSYSO:
        data['Class'][i] = 'II'
    
    for i in TransDiskYSO:
        data['Class'][i] = 'TD'
    
    for i in w4Class1YSO:
        data['Class'][i] = 'I'
    
    

#create a table
    newTable = Table()
    for name in data.colnames:
        col = Column(name = name, dtype = data[name].dtype)
        newTable.add_column(col)
#copy rows of YSO candidates from data to newTable
    for i in allYSO:
        newTable.add_row(data[int(i)])
    newFilename = '../../KoenigSelectedCatalogs/tempTaurus/part'+str(start)+'.txt'
    ascii.write(newTable, newFilename)
    start = start + 1
#This file is the section of the final catalog cooresponding to this strip of the WISE data
#All strips will then be combined using the 'Combine Koenig Strips' ipython notebook.

